In [1]:
import boto3 #comment
import os
import requests
import json

# Replace request_body with the extracted feature values
feature_values = [
    1.417779575520277, 1.2235334622079015, 1376.2808414176138, -44282.09192720655,
    1.0663144705090994, -0.016425213918793445, -0.12412711151320194, 3.010860708305122,
    6.446308650295107, 1134.2601420161861, 83.01687506148825, 1950.9720074899026, 38.33997291730212,
    98187.71575450613, 47.512356777412954, -122.29738607549102, 1162.588195657737, -15814.149590757263
]

# Constructing the JSON payload
request_body = json.dumps({ "instances": [feature_values] })

# For authentication options if running outside of domino, see
# https://docs.dominodatalab.com/en/latest/user_guide/40b91f/domino-api-authentication/
base_url = os.environ['DOMINO_API_PROXY']

model_deployment_id='672dd5af36c5431d06b18538'
endpoint_name='672dd5af36c5431d06b18538-endpoint'
region = 'us-west-2'
credentials_url = f"{base_url}/api/modelServing/v1/modelDeployments/{model_deployment_id}/credentials/INVOKE_ENDPOINT"

credentials_response = requests.get(credentials_url).json()
session = boto3.Session(aws_access_key_id=credentials_response['credentials']['AWS_ACCESS_KEY_ID'],
                        aws_secret_access_key=credentials_response['credentials']['AWS_SECRET_ACCESS_KEY'],
                        aws_session_token=credentials_response['credentials']['AWS_SESSION_TOKEN'])

sagemaker_client = session.client("sagemaker-runtime", region_name=region)

try:
    response = sagemaker_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=request_body,
        ContentType='application/json',
        Accept='application/json',
    )
    print(response['Body'].read().decode('utf-8'))
except sagemaker_client.exceptions.ModelError as e:
    print(f"error invoking endpoint: {e}")


{"predictions": [265000.0]}
